In [5]:
import pandas as pd
import re

In [6]:
df = pd.read_excel('Exlibri.xlsx', names=['signatur','jahrn','jahrz','artv','artn','artidn','besitv','besitn','beidn','verlagn','verlagidn','titel','sprache','ort','umfang','blatt','bild','platte','technik','technikidn','bemerk'], dtype='string')
df.head()

,signatur,jahrn,jahrz,artv,artn,artidn,besitv,besitn,beidn,verlagn,...,titel,sprache,ort,umfang,blatt,bild,platte,technik,technikidn,bemerk
0,1988/Bl. 750,1988,vor 1988,<NA>,<NA>,<NA>,C. G.,Schönkopf,<NA>,<NA>,...,"Exlibris-Radierung mit Flasche auf Podest, umr...",<NA>,<NA>,,290 x 198,30 x 30,45 x 45,Radierung,<NA>,<NA>
1,1981/Bl. 117,1981,vor 1981,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,Exlibris mit im Stil von Vorsatzpapier in Wabe...,ger,<NA>,<NA>,150 x 90,<NA>,<NA>,<NA>,<NA>,<NA>
2,1966/Bl. 147,1601,1601-1700,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,Exlibris mit viergeteiltes Wappenschild - Löwe...,<NA>,<NA>,<NA>,205 x 180,<NA>,185 x 170,Kupferstich,<NA>,<NA>
3,1966/Bl. 149,1701,1701 - 1800,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,Exlibris mit Sinnspruch: In aeternum non commo...,lat,<NA>,<NA>,111 x 132,92 x 111,<NA>,Kupferstich,<NA>,<NA>
4,1966/Bl. 158,1701,1701 - 1800,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,Exlibris mit Devise: Adjuvante Servatore : Mut...,<NA>,<NA>,<NA>,93 x 63,<NA>,<NA>,Kupferstich,<NA>,<NA>


In [20]:
def pica_schreiben(row):

    pica = f"""0500 Pa
0501 $bsti
0502 $bn
0503 $bnb
0600 yy
"""
    if pd.notna(row.jahrz):
        pica += f"1100 {row.jahrn}$n{row.jahrz}\n"
    else:
        pica += f"1100 {row.jahrn}\n"
    
    pica += f"""1130 TB-papier
1132 a1-analog;f1-bild;f2-blatt;e2-uv
"""
    if pd.notna(row.sprache):
        pica += f"1500 /1{row.sprache}\n"

    if pd.notna(row.artidn):
        pica += f"3010 {row.artidn}$BKünstler$4art\n"
    
    if pd.isna(row.artidn) and pd.notna(row.artn):
        pica += f"3010 {row.artn}, {row.artv}$BKünstler$4art\n"

    if pd.notna(row.beidn):
        pica += f"3010 {row.beidn}$BFrüherer Eigentümer$4fmo\n"

    if pd.isna(row.beidn) and pd.notna(row.besitn):
        pica += f"3010 {row.besitn}, {row.besitv}$BFrüherer Eigentümer$4fmo\n"

    if pd.notna(row.verlagidn):
        pica += f"3110 {row.verlagidn}\n"
    
    if pd.notna(row.verlagn):
        pica += f"3110 {row.verlagn}\n"
    
    pica += f"4000 {row.titel}"

    if pd.notna(row.artn):
        pica += f" \ von {row.artv} {row.artn}"

    if pd.notna(row.besitn):
        pica += f" ; für {row.besitv} {row.besitn}"

    if pd.notna(row.verlagn):
        pica += f" für {row.verlagn}"
    pica += f"\n"

    pica += f"Exlibris$Bobja\n"

    if pd.notna(row.ort):
        pica += f"4046 {row.ort}\n"
    else:
        pica += f"4046 [Ort nicht ermittelbar]\n"

    if pd.notna(row.umfang):
        pica += f"4060 {row.umfang}\n"
    else:
        pica += f"4060 1 Blatt\n"

    if pd.notna(row.blatt):
        blattwerte = row.blatt.split(' x ')
        pica += f"4062 {blattwerte[0]} x {blattwerte[1]} mm$b{blattwerte[0]}$h{blattwerte[1]}$4bltt\n"

    if pd.notna(row.bild):
        bildwerte = row.bild.split(' x ')
        pica += f"4062 {bildwerte[0]} x {bildwerte[1]} mm$b{bildwerte[0]}$h{bildwerte[1]}$4bild\n"  


    if pd.notna(row.platte):
        plattenwerte = row.platte.split(' x ')
        pica += f"4062 {plattenwerte[0]} x {plattenwerte[1]} mm$b{plattenwerte[0]}$h{plattenwerte[1]}$4???\n"

    pica += f"""4105 !1060610671!
5590 [Objektgattung]
5590 !04015985X!
"""

    if pd.notna(row.technikidn):
        pica +=f"5591 [Technik]\n5591 {row.technikidn}\n"
    
    pica +=f"""7001 x
4800 !009033645!
6710 !1126610925!
7100 {row.signatur} @ m
7109 !!DBSM/S!! ; {row.signatur}
"""
    if pd.notna(row.bemerk):
        pica += f"8034 {row.bemerk}\n"

    pica += f"8598 {row.signatur}\n"
        

    return pica


    




In [21]:
for index, row in df.sample(15).iterrows():
    print(pica_schreiben(row))

0500 Pa
0501 $bsti
0502 $bn
0503 $bnb
0600 yy
1100 1958
1130 TB-papier
1132 a1-analog;f1-bild;f2-blatt;e2-uv
3010 Abad, R.$BKünstler$4art
3010 !142433713!$BFrüherer Eigentümer$4fmo
4000 Exlibris - Strichätzung nach einem Holzschnitt \ von R. Abad ; für Enrique Saez Fernandez Casariego
Exlibris$Bobja
4046 [Ort nicht ermittelbar]
4060 1 Blatt
4105 !1060610671!
5590 [Objektgattung]
5590 !IDN Exlibris
7001 x
4800 !BIC DBSM!
6710 !1126610925!
7100 1960/Bl. 30 @ m
7109 !!DBSM/S!! ; 1960/Bl. 30
8598 1960/Bl. 30

0500 Pa
0501 $bsti
0502 $bn
0503 $bnb
0600 yy
1100 1979
1130 TB-papier
1132 a1-analog;f1-bild;f2-blatt;e2-uv
3010 !1050402170!$BKünstler$4art
3010 !104167807X!$BFrüherer Eigentümer$4fmo
4000 Exlibris - Holzschnitt - mit Specht am Baumstamm, von Vögeln umgeben ; handsigniert rechts unten \ von Marie-Louise Albessart ; für K.-H. Anger
Exlibris$Bobja
4046 Baumont
4060 1 Blatt
4062 102 x 80 mm$b102$h80$4bild
4105 !1060610671!
5590 [Objektgattung]
5590 !IDN Exlibris
7001 x
4800 !BIC DBSM!


IndexError: list index out of range